## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-06-18-19-40-00 +0000,wsj,Fed officials continued to pencil in two rate ...,https://www.wsj.com/economy/central-banking/fe...
1,2025-06-18-19-36-52 +0000,nyt,Hurricane Erick Nears Mexico’s Southern Coast ...,https://www.nytimes.com/2025/06/18/weather/hur...
2,2025-06-18-19-31-00 +0000,bbc,Karen Read found not guilty of murder of her p...,https://www.bbc.com/news/articles/ckgnk27z504o
3,2025-06-18-19-26-14 +0000,bbc,Watch: Trump installs enormous US flags at the...,https://www.bbc.com/news/videos/ckg3n1yl5k8o
4,2025-06-18-19-14-43 +0000,bbc,Eleven killed by Israeli fire while seeking ai...,https://www.bbc.com/news/articles/c7841705x18o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


C:\Users\CJ\AppData\Local\Temp\ipykernel_36116\2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,5
58,bank,5
102,iran,4
26,police,3
89,family,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-06-18-18-25-00 +0000,wsj,Bank of Canada Gov. Tiff Macklem said it is po...,https://www.wsj.com/articles/canada-core-cpi-m...,31
9,2025-06-18-17-51-00 +0000,wsj,‘Tariffs have lowered our exports and weighed ...,https://www.wsj.com/articles/bank-of-canada-to...,19
0,2025-06-18-19-40-00 +0000,wsj,Fed officials continued to pencil in two rate ...,https://www.wsj.com/economy/central-banking/fe...,19
40,2025-06-18-08-16-00 +0000,wsj,Sweden’s Central Bank Cuts Rate Amid Economic ...,https://www.wsj.com/articles/swedens-central-b...,18
13,2025-06-18-16-37-34 +0000,bbc,Trump's Iran dilemma exposes bitter split in p...,https://www.bbc.com/news/articles/c1wp72224jzo,17


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,31,2025-06-18-18-25-00 +0000,wsj,Bank of Canada Gov. Tiff Macklem said it is po...,https://www.wsj.com/articles/canada-core-cpi-m...
0,19,2025-06-18-19-40-00 +0000,wsj,Fed officials continued to pencil in two rate ...,https://www.wsj.com/economy/central-banking/fe...
13,17,2025-06-18-16-37-34 +0000,bbc,Trump's Iran dilemma exposes bitter split in p...,https://www.bbc.com/news/articles/c1wp72224jzo
11,15,2025-06-18-17-20-26 +0000,bbc,'They sang about my family's death': Father's ...,https://www.bbc.com/news/articles/c7vr51pzvm0o
37,15,2025-06-18-09-00-00 +0000,wsj,The U.S.’s publicly held debt is nearing 100% ...,https://www.wsj.com/politics/policy/us-budget-...
10,13,2025-06-18-17-34-48 +0000,bbc,"'No-bra, no-exam' rule at Nigerian university ...",https://www.bbc.com/news/articles/c2d0p3z9grko
17,12,2025-06-18-15-36-39 +0000,bbc,Analysis: Two issues dominating Parliament and...,https://www.bbc.com/news/articles/cm20y9jl77do
29,12,2025-06-18-12-54-00 +0000,wsj,The Federal Reserve’s dot-plot rate projection...,https://www.wsj.com/economy/central-banking/fe...
2,11,2025-06-18-19-31-00 +0000,bbc,Karen Read found not guilty of murder of her p...,https://www.bbc.com/news/articles/ckgnk27z504o
45,11,2025-06-18-01-23-18 +0000,bbc,MP once on benefits calls cuts 'brutal' - but ...,https://www.bbc.com/news/articles/cvgqg7ww014o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
